In [ ]:

"""
Low-Volatility Rotating Portfolio (5 slots)
------------------------------------------
* Universe: 10 US large-cap stocks
* Signal:   “top-5 lowest 6-month σ AND price > 200-DMA”  (re-evaluated daily)
* Weighting: equal among the active names each day
* Risk:     skip entries when VIX > 30   |   stop each leg at –10 %
* Engine:   Pure pandas (Backtesting.py runs single-ticker stats only)
            → final portfolio P/L is stitched from per-ticker equity curves.
"""
#https://youtu.be/86jcGtoLIHM?si=G3-5Hlt44-XU_YNy
import yfinance as yf
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy

# ----------------------------------------------------------------------
# 1 ─ Download price data and VIX
# ----------------------------------------------------------------------
TICKERS = [
    # mega-tech & comm services
    'AAPL','MSFT','GOOGL','AMZN','META','NVDA','TSLA','NFLX','CRM','ORCL',

    # diversified conglomerates / Berkshire
    'BRK-B',

    # money-center & card networks
    'JPM','BAC','WFC','MA','V','USB',

    # healthcare, biopharma, med-tech
    'JNJ','UNH','LLY','PFE','MRK','ABBV','ABT','TMO','GILD',

    # consumer staples / discretionary
    'WMT','COST','PG','KO','PEP','HD','MCD','NKE','DIS',

    # energy & materials
    'XOM','CVX','LIN',

    # industrials & multi-sector
    'HON','DHR','ACN',

    # semis & hardware
    'INTC','AMD','AVGO','TXN','QCOM','CSCO'
]
START, END = "2015-01-01", "2025-06-20"

# Adjusted close prices (splits/divs handled automatically)
prices = (yf.download(TICKERS, START, END, progress=False)['Close'])#.dropna(how="all"))
vix = (yf.download("^VIX", START, END, progress=False)['Close'].reindex(prices.index).ffill())
f:\Python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10788\685991413.py:48: FutureWarning: YF.download() has changed argument auto_adjust default to True
  prices = (yf.download(TICKERS, START, END, progress=False)['Close'])#.dropna(how="all"))
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10788\685991413.py:49: FutureWarning: YF.download() has changed argument auto_adjust default to True
  vix = (yf.download("^VIX", START, END, progress=False)['Close'].reindex(prices.index).ffill())
len(TICKERS)
47
# ----------------------------------------------------------------------
# 2 ─ Pre-compute indicators
# ----------------------------------------------------------------------

# === Strategy parameters ===
VOL_WINDOW      = 90          # look-back (trading days) for volatility
MA_WINDOW       = 150          # look-back for trend filter (simple MA)
MAX_POSITIONS   = 5           # maximum number of assets held
STOP_LOSS_PCT   = -0.10       # stop loss (-10 %)
VIX_THRESHOLD   = 25        # optional VIX filter (not used below)
MAX_HOLD_DAYS   = 90          # maximum holding period

# === Indicator calculations ===
daily_returns  = prices.pct_change()
annual_vol     = daily_returns.rolling(VOL_WINDOW).std() * np.sqrt(252)  # annualised σ
trend_ma       = prices.rolling(MA_WINDOW).mean()                        # 20-day SMA

# ───────────────────────────────────────────────────────────────
# 3.  Asset-selection helper
# ───────────────────────────────────────────────────────────────
def pick_low_volatility_assets(
        vol_row:   pd.Series,
        price_row: pd.Series,
        ma_row:    pd.Series,
        n: int = MAX_POSITIONS
) -> pd.Series:
    """
    Decide which symbols enter the portfolio on *one* date.

    Steps
    -----
    1. **Trend filter**  – only trade symbols whose price is above their
       moving average (up-trend).
    2. **Volatility rank** – among those, rank by realised volatility
       (lower is better).
    3. **Selection** – return a boolean Series that is *True* for the
       `n` lowest-volatility symbols, *False* otherwise.

    Parameters
    ----------
    vol_row   : annualised volatility for this date (one row of `annual_vol`)
    price_row : prices for this date (one row of `prices`)
    ma_row    : moving averages for this date (one row of `trend_ma`)
    n         : how many symbols to pick (default: MAX_POSITIONS)
    """
    # 1. Trend filter
    in_uptrend = price_row > ma_row

    # 2. Rank (NaNs for out-of-trend symbols are ignored)
    ranked_vol = vol_row.where(in_uptrend).rank(method="first")

    # 3. Select the `n` smallest ranks
    winners = ranked_vol.nsmallest(n).index

    return pd.Series(price_row.index.isin(winners), index=price_row.index)


# ───────────────────────────────────────────────────────────────
# 4.  Build the daily signal matrix
#     True  → symbol qualifies for today’s low-vol basket
# ───────────────────────────────────────────────────────────────
signals = pd.DataFrame(
    [
        pick_low_volatility_assets(
            annual_vol.loc[date],
            prices.loc[date],
            trend_ma.loc[date]
        )
        for date in prices.index
    ],
    index=prices.index,
    columns=prices.columns,
).astype(bool)
signals
Ticker	AAPL	ABBV	ABT	ACN	AMD	AMZN	AVGO	BAC	BRK-B	COST	...	QCOM	TMO	TSLA	TXN	UNH	USB	V	WFC	WMT	XOM
Date																					
2015-01-02	True	True	True	True	True	False	False	False	False	False	...	False	False	False	False	False	False	False	False	False	False
2015-01-05	True	True	True	True	True	False	False	False	False	False	...	False	False	False	False	False	False	False	False	False	False
2015-01-06	True	True	True	True	True	False	False	False	False	False	...	False	False	False	False	False	False	False	False	False	False
2015-01-07	True	True	True	True	True	False	False	False	False	False	...	False	False	False	False	False	False	False	False	False	False
2015-01-08	True	True	True	True	True	False	False	False	False	False	...	False	False	False	False	False	False	False	False	False	False
...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...
2025-06-12	False	False	True	False	False	False	False	False	False	False	...	False	False	False	False	False	False	False	False	False	False
2025-06-13	False	False	True	False	False	False	False	False	False	False	...	False	False	False	False	False	False	False	False	False	False
2025-06-16	False	False	True	False	False	False	False	False	True	False	...	False	False	False	False	False	False	False	False	False	False
2025-06-17	False	False	True	False	False	False	False	False	False	False	...	False	False	False	False	False	False	False	False	False	False
2025-06-18	False	False	True	False	False	False	False	False	False	False	...	False	False	False	False	False	False	True	False	False	False
2631 rows × 47 columns

signals.sum().sort_values(ascending=False)
Ticker
PEP      1488
KO       1376
MCD      1184
PG       1118
BRK-B    1063
JNJ      1051
WMT       677
COST      588
MRK       383
HON       366
PFE       364
V         293
ABBV      279
ACN       274
ABT       270
DHR       238
USB       218
LIN       208
AAPL      165
DIS       158
AMD       154
MA        151
UNH       130
XOM       129
HD        128
JPM       112
CSCO      107
GILD      101
LLY        96
AMZN       91
ORCL       60
NKE        23
GOOGL      22
TMO        20
NFLX       18
CVX        18
WFC        11
META        8
MSFT        6
TSLA        5
NVDA        2
CRM         2
BAC         0
QCOM        0
TXN         0
AVGO        0
INTC        0
dtype: int64
# ----------------------------------------------------------------------
# 3 ─ Single-stock back-test (Backtesting.py) following its own Signal
# ----------------------------------------------------------------------
from backtesting import Strategy
import math, numpy as np

class SignalFollower(Strategy):
    # ────────── parameters you may tweak ──────────
    n_slots      = MAX_POSITIONS      # max simultaneous holdings
    stop_pct     = STOP_LOSS_PCT   # per-trade stop (%); None → disable
    vix_level    = VIX_THRESHOLD    # e.g. 30 blocks new entries; None → ignore
    max_holding  = MAX_HOLD_DAYS      # force-close after N bars
    # ------------------------------------------------

    def init(self):
        self.bar_clock     = 0        # global bar counter
        self.entry_bar_idx = None     # remembers when current pos opened

    # ------------------------------------------------
    def next(self):
        self.bar_clock += 1

        # 1) stop-loss
        if self.stop_pct is not None:
            for tr in self.trades:            # only open trades
                if tr.pl_pct < self.stop_pct:
                    tr.close()

        # 2) age-out after max_holding bars
        if self.position and self.entry_bar_idx is not None:
            if self.bar_clock - self.entry_bar_idx >= self.max_holding:
                self.position.close()
                self.entry_bar_idx = None

        # 3) entry / exit
        in_sig = bool(self.data.Signal[-1])
        vix_block = (self.vix_level is not None and
                     self.data.VIX[-1] > self.vix_level)

        if in_sig and not self.position and not vix_block:
            alloc_cash = self._broker._cash / self.n_slots   # ← fixed here
            price      = self.data.Close[-1]
            shares     = int(alloc_cash / price)
            if shares < 1:
                return
            self.buy(size=shares)
            self.entry_bar_idx = self.bar_clock

        elif (not in_sig) and self.position:
            self.position.close()
            self.entry_bar_idx = None

    # ------------------------------------------------
    def stop(self):
        # ensure last open counts in "# Trades"
        if self.position:
            self.position.close()
# ----------------------------------------------------------------------
# 4 ─ Run one Backtest per ticker, collect stats & equity curves
# ----------------------------------------------------------------------
stats, curves = {}, []

for ticker in TICKERS:
    sig = signals[ticker].reindex(prices.index).ffill()

    df = pd.DataFrame({
        'Close' : prices[ticker].squeeze(),
        'Signal': signals[ticker].squeeze(),
        'VIX'   : vix.squeeze()
    })
    
    df = (df.assign(Open=df['Close'], High=df['Close'], Low=df['Close'])
            [['Open','High','Low','Close','Signal','VIX']]
            .dropna())

    bt = Backtest(df, SignalFollower, cash=100000, commission=0.001)
    s  = bt.run()
    stats[ticker] = s
    curves.append(s['_equity_curve']['Equity'].rename(ticker))
# ------------------------------------------------------------------
# 5 ─ Build the portfolio equity curve (sum $-P/L across slots)
# ------------------------------------------------------------------
INIT_CAP  = 100_000          # total capital of the book
# SLOT_CAP  = INIT_CAP / MAX_POSITIONS

equity_df = pd.concat(curves, axis=1)          #   one col per ticker
# Each column starts at 100k but only ever puts 20k at risk.
pl_df     = equity_df.sub(INIT_CAP)            #   isolate slot P/L ($)
total_pl  = pl_df.sum(axis=1)                  #   add the five slots
PORT_EQ   = INIT_CAP + total_pl                #   true portfolio equity
port_ret  = PORT_EQ.pct_change().fillna(0)  
# ──────────────────────────────────────────────────────────────────────
# 6 ─ Summary table
# ──────────────────────────────────────────────────────────────────────
def sharpe(r):
    return r.mean() / r.std() * np.sqrt(252)

summary = pd.DataFrame({
    "Return [%]":  (PORT_EQ.iloc[-1] / PORT_EQ.iloc[0] - 1) * 100,
    "Sharpe Ratio": sharpe(port_ret),
    "Max DD [%]": 100 * (PORT_EQ.cummax() - PORT_EQ).max()
                       / PORT_EQ.cummax().max(),
    "# Trades":   signals.diff().abs().sum().sum()
}, index=["Portfolio"])

print("=== Portfolio summary ===")
print(summary.round(2))
=== Portfolio summary ===
           Return [%]  Sharpe Ratio  Max DD [%]  # Trades
Portfolio       78.26          0.63        8.93      1954
import plotly.graph_objects as go
import matplotlib.pyplot as plt

# 1) Gather individual curves into one DataFrame
equity_curves = pd.concat(curves, axis=1)        # cols = tickers
equity_curves.columns.name = "Symbol"

# 2) Plot
fig, ax = plt.subplots(figsize=(12, 6))

# -- Individual assets (thin, semi-transparent)
equity_curves.plot(ax=ax, lw=1.0, alpha=0.7)

# -- Portfolio total (thicker, darker line)
PORT_EQ.plot(ax=ax, color="black", lw=2.5, label="Portfolio total")

# 3) Cosmetics
ax.set_title("Equity Curves – Individual Assets vs. Portfolio")
ax.set_ylabel("Equity ($)")
ax.grid(True, alpha=0.3)
ax.legend(loc="upper left", ncol=2)

plt.tight_layout()
plt.show()

import plotly.graph_objects as go

fig = go.Figure()

# Individual curves
for col in equity_curves.columns:
    fig.add_trace(go.Scatter(
        x=equity_curves.index, y=equity_curves[col],
        mode="lines", name=col, line=dict(width=3)))

# Portfolio total
fig.add_trace(go.Scatter(
    x=PORT_EQ.index, y=PORT_EQ,
    mode="lines", name="Portfolio total", line=dict(width=6, color="white")))

fig.update_layout(
    title="Equity Curves – Individual Assets vs. Portfolio",
    xaxis_title="Date", yaxis_title="Equity ($)",
    plot_bgcolor='black',
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    legend=dict(orientation="h", yanchor="bottom", y=1.02),
    width=1200, height=800)  # Updated figure size

fig.show()
# ────────────────────────────────────────────────────────────────
# 7 ─ Per-ticker headline metrics + diagnostics
# ────────────────────────────────────────────────────────────────
wanted = ["Return [%]", "Sharpe Ratio", "Max. Drawdown [%]", "# Trades"]

# ①  Pull headline numbers from Backtesting.py
per_ticker = (
    pd.DataFrame({t: {m: s[m] for m in wanted}
                  for t, s in stats.items()})
      .T
)

# ②  Days the stock qualified for the basket   (signal == True)
per_ticker["Days in basket"] = signals.sum()

# ③  Days that ALSO passed the VIX filter
vix_ok   = (vix <= VIX_THRESHOLD).to_numpy().reshape(-1, 1)   # (n_days, 1)
eligible = signals & vix_ok                               # DataFrame[bool]
per_ticker["Days VIX≤limit"] = eligible.sum()

# ④  Distinct entry SET-UPS  (False→True transitions of ‘eligible’)
entry_setups = (eligible & ~eligible.shift(fill_value=False)).sum()
per_ticker["Entry setups"] = entry_setups

# ⑤  Actual holding days   (duration of open trades)
held = {}
for t in TICKERS:
    trades = stats[t]["_trades"]
    held[t] = 0 if trades.empty else (trades["ExitBar"] - trades["EntryBar"] + 1).sum()
per_ticker["Days held"] = pd.Series(held)

# ⑥  Arrange columns & sort
cols = wanted + ["Entry setups", "Days held", "Days VIX≤limit", "Days in basket"]
per_ticker = per_ticker[cols].sort_values("Return [%]", ascending=False)

print("\n=== Single-ticker stats (with diagnostics) ===")
per_ticker.round(2)
=== Single-ticker stats (with diagnostics) ===
Return [%]	Sharpe Ratio	Max. Drawdown [%]	# Trades	Entry setups	Days held	Days VIX≤limit	Days in basket
BRK-B	19.20	0.82	-3.36	64.0	69	1071	972	1063
DHR	15.34	0.33	-1.61	24.0	26	207	178	238
PG	9.16	0.42	-4.34	68.0	83	1161	1009	1118
COST	8.72	0.47	-3.55	39.0	44	547	460	588
HON	6.98	0.54	-4.16	38.0	38	397	354	366
WMT	6.17	0.35	-4.83	38.0	40	621	571	677
KO	5.14	0.22	-4.45	57.0	66	1379	1241	1376
MCD	4.88	0.22	-4.53	55.0	71	1160	1017	1184
ABT	4.78	0.39	-1.78	13.0	14	261	250	270
ACN	3.87	0.27	-2.12	23.0	24	289	260	274
AMZN	3.39	0.30	-1.81	3.0	4	37	5	91
AAPL	3.12	0.24	-3.08	4.0	2	155	154	165
DIS	2.71	0.40	-1.84	13.0	13	171	158	158
ABBV	2.67	0.18	-3.31	13.0	15	225	209	279
MRK	2.63	0.17	-3.12	31.0	41	358	272	383
NKE	2.17	0.34	-0.50	1.0	2	14	12	23
GILD	1.97	0.22	-1.78	9.0	12	75	29	101
LLY	1.61	0.33	-1.39	8.0	8	47	39	96
JPM	1.55	0.16	-2.59	10.0	10	119	109	112
ORCL	0.97	0.10	-1.94	10.0	12	69	50	60
PEP	0.68	0.03	-5.68	46.0	57	1460	1314	1488
HD	0.43	0.05	-3.01	23.0	24	140	115	128
LIN	0.17	0.01	-3.61	23.0	24	229	198	208
TMO	0.17	0.05	-0.78	2.0	2	13	11	20
MSFT	0.03	0.16	-0.02	1.0	1	2	1	6
TSLA	0.00	NaN	-0.00	0.0	0	0	0	5
NVDA	0.00	NaN	-0.00	0.0	0	0	0	2
INTC	0.00	NaN	-0.00	0.0	0	0	0	0
AVGO	0.00	NaN	-0.00	0.0	0	0	0	0
TXN	0.00	NaN	-0.00	0.0	0	0	0	0
QCOM	0.00	NaN	-0.00	0.0	0	0	0	0
BAC	0.00	NaN	-0.00	0.0	0	0	0	0
NFLX	0.00	NaN	-0.00	0.0	0	0	0	18
CRM	0.00	NaN	-0.00	0.0	0	0	0	2
META	-0.40	0.00	-0.63	1.0	1	5	4	8
GOOGL	-0.60	0.00	-1.35	1.0	1	16	15	22
CSCO	-1.06	0.00	-3.09	18.0	20	122	101	107
PFE	-1.60	0.00	-5.50	28.0	30	356	326	364
CVX	-1.71	0.00	-1.71	4.0	4	15	11	18
WFC	-2.18	0.00	-2.32	1.0	1	12	11	11
V	-2.34	0.00	-4.26	30.0	31	316	285	293
USB	-2.36	0.00	-5.86	23.0	23	236	213	218
UNH	-2.60	0.00	-3.27	15.0	16	101	82	130
XOM	-2.77	0.00	-3.59	8.0	8	125	117	129
AMD	-3.05	0.00	-12.90	5.0	1	149	149	154
JNJ	-4.42	0.00	-9.90	56.0	69	968	842	1051
MA	-5.16	0.00	-5.16	23.0	24	170	146	151
import plotly.graph_objects as go

fig = go.Figure()

# Individual curves
for col in equity_curves.columns:
    fig.add_trace(go.Scatter(
        x=equity_curves.index, y=equity_curves[col],
        mode="lines", name=col, line=dict(width=3)))

fig.update_layout(
    title="Equity Curves – Individual Assets vs. Portfolio",
    xaxis_title="Date", yaxis_title="Equity ($)",
    plot_bgcolor='black',
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    legend=dict(orientation="h", yanchor="bottom", y=1.02),
    width=1200, height=800)  # Updated figure size

fig.show()
vix.describe()
Ticker	^VIX
count	2631.000000
mean	18.404629
std	7.284520
min	9.140000
25%	13.435000
50%	16.540001
75%	21.450000
max	82.690002
 